# MAIN

In [2]:

%load_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path
import pandas as pd 
import logging
from src.data.get_data import get_all_input_data #get_data_reco_custdim_spdim , get_kyc_customers, get_customer_score
from datetime import datetime, timedelta 
from src.data.preprocessing import preprocessing
from src.routing.ValhallaManager import ValhallaManager
from src.data.data_filter import data_filter 
from src.main import run_push_recommendation 
from src.utils_and_postprocessing.utils import setup_logger, setup_directories, list_files_in_directory, load_feather_inputs

from src.data.export import export_data 
from src.data.get_connection import get_connection
from src.clustering.evaluate_cluster import evaluate_unsupervised_clustering
from src.utils_and_postprocessing.utils import cluster_summary_and_selection, postprocess_selected_trip
from src.utils_and_postprocessing.run_clustering_and_routing import create_and_plot_route, create_cluster_trip_optroute 

In [3]:
%reload_ext autoreload

### Logger Set-up

In [4]:
logger = setup_logger(__name__, log_file='test.log')

logger.info("This is an info message")
logger.warning("This is a warning")

INFO: This is an info message


### Parameter Setup

In [5]:
# Constants for directory structure
# BASE_DIR = Path(__file__).resolve().parent
BASE_DIR = Path('').resolve()#.parent 
INDEX_HTML = 'index.html'
CURRENT_DATE = datetime.today().date() + timedelta(days=1)

INPUT_DIR, SELECTED_TRIP_PATH, ALL_CLUSTER_PATH, LOCAL_EXCEL_PATH = setup_directories(base_dir = BASE_DIR, current_date = CURRENT_DATE, logger = logger) 

INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-03
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/cluster_map/2025-07-03
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/excel_docs/2025-07-03
INFO: Copied index.html to /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-03/index.html
INFO: Copied index.html to /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/cluster_map/2025-07-03/index.html


In [9]:
# Instantiate the manager
valhalla_manager = ValhallaManager(logger=logger)

# Start the server
valhalla_manager.start_valhalla()

# Check valhalla status
valhalla_manager.check_valhalla_status()

Starting Valhalla container...


 Network valhalla_nigeria_project_default  Creating
 Network valhalla_nigeria_project_default  Created
 Container valhalla_nigeria_project-valhalla-1  Creating
 Container valhalla_nigeria_project-valhalla-1  Created
 Container valhalla_nigeria_project-valhalla-1  Starting
 Container valhalla_nigeria_project-valhalla-1  Started

Waiting 10 seconds for Valhalla to initialize...
Valhalla container started.


True

## Main Function

## MAIN 1

In [6]:
# Get input data
df_customer_sku_recommendation_raw, df_customer_dim_with_affinity_score_raw, \
    df_stockpoint_dim_raw, df_kyc_customer, df_customer_score = get_all_input_data(logger=logger,
                                                                                    save_local = False,
                                                                                    input_dir_path = None, # Added optional input_dir_path
                                                                                    reload_recommendation_data = False # Added parameter to control data reloading
                                                                                    )

INFO: Executing stored procedure(s) to fetch data...


INFO: Data fetch complete.
INFO: --- Fetched DataFrames Shapes ---
INFO: Customer SKU Recommendation: (135417, 13)
INFO: Customer Dimension with Affinity Score: (10908, 22)
INFO: Stockpoint Dimension: (72, 4)
INFO: ---------------------------------

INFO: Connecting to database and fetching KYC customer data...
INFO: Successfully fetched KYC customer data. Shape: (343225, 57)
INFO: KYC customers DataFrame shape: (343225, 57)
INFO: Connecting to database and fetching customer score data...
INFO: EXEC usp_GetPushCustomersScore
INFO: Successfully fetched customer score data. Shape: (27772, 15)
INFO: Customer scores DataFrame shape: (27772, 15)


In [7]:
# # # Development: save the inputs to feather for quick access
# # Load all feather files
# try:
#     dfs = load_feather_inputs(input_dir=INPUT_DIR, logger=logger)

#     # Unpack individual DataFrames
#     df_customer_sku_recommendation_raw = dfs['df_customer_sku_recommendation_raw']
#     df_customer_dim_with_affinity_score_raw = dfs['df_customer_dim_with_affinity_score_raw']
#     df_stockpoint_dim_raw = dfs['df_stockpoint_dim_raw']
#     df_kyc_customer = dfs['df_kyc_customer']
#     df_customer_score = dfs['df_customer_score']

# except FileNotFoundError as e:
#     logger.error("Missing required input file. Exiting pipeline.")
#     raise
# except Exception as e:
#     logger.error("Failed to load input data due to unexpected error.")
#     raise


In [7]:
# Preprocessing
df_customer_sku_recommendation, df_master_customer_dim, df_stockpoint_dim = preprocessing(df_customer_sku_recommendation_raw, 
                                                                                            df_customer_dim_with_affinity_score_raw, 
                                                                                            df_stockpoint_dim_raw,
                                                                                            df_customer_score,
                                                                                            df_kyc_customer)

In [9]:
# # stock_point_id = 1647394 #
# stock_point_id =  1647113
# stock_point_name = df_stockpoint_dim.query(f'Stock_Point_ID == {1647394}')['Stock_point_Name'].iloc[0] 

## Data Filter - Testing 
# df_sku_rec, df_customer_dim, df_stockpoint = data_filter(df_customer_sku_recommendation, 
#                                                                     df_master_customer_dim, 
#                                                                     df_stockpoint_dim, 
#                                                                     stockpoint_id = stock_point_id,  
#                                                                     sku_recency = 7, customer_recency = 60, number_recommendation = 10,
#                                                                     estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
#                                                                     exclude_recency_customer = 4)

# res_dict = run_push_recommendation(df_customer_sku_recommendation, 
#                             df_master_customer_dim, 
#                             df_stockpoint_dim, 
#                             stock_point_id,
#                             stock_point_name,
#                             sku_recency = 7, 
#                             customer_recency = 60, number_recommendation = 5, 
#                             estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
#                             exclude_recency_customer = 4,
#                             max_customers_per_route=20,
#                             max_volume_per_route=300,
#                             max_distance_km = 40,
#                             sel_trip_cluster = 5,
#                             min_ncust_per_cluster = 5,
#                             clustering_method = 'divisive',
#                             skip_route_optimization = False,
#                             save_to_disk = False,
#                             logger=logger)

In [10]:
ALL_STOCKPOINTS_RESULT = {}
for index, row in df_stockpoint_dim.iterrows():
    # if index == 12:
    # if index == 5:
    stock_point_id =  row['Stock_Point_ID']
    stock_point_name = row['Stock_point_Name']
    print(f'{index}/{len(df_stockpoint_dim)} \nStock Point ID: {stock_point_id} || Stock Point Name: {stock_point_name}')  # Access by column name

    res_dict = run_push_recommendation(df_customer_sku_recommendation, 
                            df_master_customer_dim, 
                            df_stockpoint_dim, 
                            stock_point_id,
                            stock_point_name,
                            sku_recency = 7, 
                            customer_recency = 60, number_recommendation = 5, 
                            estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                            exclude_recency_customer = 4,
                            max_customers_per_route=20,
                            max_volume_per_route=300,
                            max_distance_km = 40,
                            sel_trip_cluster = 5,
                            min_ncust_per_cluster = 5,
                            clustering_method = 'divisive',
                            skip_route_optimization = False,
                            save_to_disk = True,
                            # Global variables
                            valhalla_manager = valhalla_manager,
                            CURRENT_DATE = CURRENT_DATE,
                            SELECTED_TRIP_PATH = SELECTED_TRIP_PATH,
                            ALL_CLUSTER_PATH = ALL_CLUSTER_PATH,
                            LOCAL_EXCEL_PATH = LOCAL_EXCEL_PATH,
                            logger=logger)
    
    ALL_STOCKPOINTS_RESULT[stock_point_name] = res_dict

INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,
INFO: ================================================================================
INFO: 
1. Loading Data...
INFO: ----------------------------------------
INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Ado Odo/Ota Ogun - Prince Tunadek, StockPointID: 1647401,
INFO: ================================================================================


0/72 
Stock Point ID: 1647128 || Stock Point Name: OmniHub Obio Akpor Rivers - Rivoc
Total Quantity before filter: 25
Total Quantity: 25
Total Number of Customers before filter: 1
Total Number of Customers: 1
1/72 
Stock Point ID: 1647401 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Prince Tunadek


INFO: 
1. Loading Data...
INFO: ----------------------------------------


INFO: ✓ Loaded 19 SKU recommendations
INFO: ✓ Loaded 5 customer records
INFO: ✓ Loaded 1 stock points
INFO: ✓ Route optimizer initialized
INFO: 
3. Generating Optimized Routes...
INFO: ----------------------------------------


Total Quantity before filter: 308
Total Quantity: 81
Total Number of Customers before filter: 21
Total Number of Customers: 5


...

64/72 
Stock Point ID: 1647420 || Stock Point Name: OmniHub Ojo Lagos - Barka Agro 3
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
65/72 
Stock Point ID: 1647421 || Stock Point Name: OmniHub Ikpoba Okha Edo - Real Care
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
66/72 
Stock Point ID: 1647422 || Stock Point Name: OmniHub Oshodi Isolo Lagos - Fabb


INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Keffi Nasarawa - Donsam, StockPointID: 1647425,
INFO: ================================================================================
INFO: 
1. Loading Data...
INFO: ----------------------------------------
INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Kuje/Gwagwalada/Abaji Abuja - Deschek, StockPointID: 1647434,
INFO: ================================================================================
INFO: 
1. Loading Data...
INFO: ----------------------------------------
INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Egor Edo - 10TH Gear, StockPointID: 1647436,
INFO: ============

Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
67/72 
Stock Point ID: 1647425 || Stock Point Name: OmniHub Keffi Nasarawa - Donsam
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
68/72 
Stock Point ID: 1647434 || Stock Point Name: OmniHub Kuje/Gwagwalada/Abaji Abuja - Deschek
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
69/72 
Stock Point ID: 1647436 || Stock Point Name: OmniHub Egor Edo - 10TH Gear


INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Ado Odo/Ota Ogun - Azacram, StockPointID: 1647437,
INFO: ================================================================================
INFO: 
1. Loading Data...
INFO: ----------------------------------------
INFO: ================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Ado Odo/Ota Ogun - Soffy's Concept, StockPointID: 1647438,
INFO: ================================================================================
INFO: 
1. Loading Data...
INFO: ----------------------------------------


Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
70/72 
Stock Point ID: 1647437 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Azacram
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
71/72 
Stock Point ID: 1647438 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Soffy's Concept
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0


## Fix Map

In [18]:
all_spid_list = df_stockpoint_dim['Stock_Point_ID'].to_list()
selected_trip_spid_list = [int(spid) for spid in list_files_in_directory(SELECTED_TRIP_PATH) if not spid.startswith('index')]
all_cluster_spid_list = [int(spid) for spid in list_files_in_directory(ALL_CLUSTER_PATH) if not spid.startswith('index')]

unmapped_selected_trip_spid_list = list(set(all_spid_list) - set(selected_trip_spid_list))
unmapped_all_cluster_spid_list = list(set(all_spid_list) - set(all_cluster_spid_list))

In [19]:
print(f"All Stock Points: {len(all_spid_list)}")
print(f"All Stock Points: {len(selected_trip_spid_list)}")
print(f"All Stock Points: {len(all_cluster_spid_list)}")
print(f"All Stock Points: {len(unmapped_selected_trip_spid_list)}")
print(f"All Stock Points: {len(unmapped_all_cluster_spid_list)}")

All Stock Points: 72
All Stock Points: 72
All Stock Points: 72
All Stock Points: 0
All Stock Points: 0


In [16]:
# Copy index.html files from source directories to date-based subdirectories
source_cluster_map_index = BASE_DIR / 'html' / 'default_cluster_map_index.html'
source_selected_trip_index = BASE_DIR / 'html' / 'default_selected_cluster_map_index.html' 
        
if len(unmapped_all_cluster_spid_list) > 0:
    try:
        if source_cluster_map_index.exists():
            for spid in unmapped_all_cluster_spid_list:
                spid_path = ALL_CLUSTER_PATH / f'{str(spid)}.html'       
                shutil.copy2(source_cluster_map_index, spid_path)
                logger.debug(f"Copied index.html to {ALL_CLUSTER_PATH / str(spid)}.html")
        else:
            logger.warning(f"Source index.html not found at {source_selected_trip_index}")
    except Exception as e:
        logger.error(f"Failed to copy index.html to {ALL_CLUSTER_PATH}: {str(e)}")
        
        
# Copy index.html for selected_trip_map
if len(unmapped_selected_trip_spid_list) > 0:
    try:
        if source_selected_trip_index.exists():
            for spid in unmapped_selected_trip_spid_list:
                spid_path = SELECTED_TRIP_PATH / f'{str(spid)}.html'       
                shutil.copy2(source_selected_trip_index, spid_path)
                logger.debug(f"Copied index.html to {SELECTED_TRIP_PATH / INDEX_HTML}")
        else:
            logger.warning(f"Source index.html not found at {source_selected_trip_index}")
    except Exception as e:
        logger.error(f"Failed to copy index.html to {SELECTED_TRIP_PATH}: {str(e)}")

             

In [14]:
# unmapped_selected_trip_spid_list

## Export to DB

In [15]:
from src.data.export2db import RecommendationProcessor
from src.data.get_connection import get_connection

In [16]:
# Simple usage
# main(ALL_STOCKPOINTS_RESULT, CURRENT_DATE, get_connection)

# Or use the processor directly
processor = RecommendationProcessor(get_connection)
processor.process(ALL_STOCKPOINTS_RESULT, CURRENT_DATE)

In [18]:
CURRENT_DATE

datetime.date(2025, 7, 3)

# Clean Up

In [17]:
# Stop the server when done
valhalla_manager.stop_valhalla()

Stopping Valhalla container...
 Container valhalla_nigeria_project-valhalla-1  Stopping
 Container valhalla_nigeria_project-valhalla-1  Stopped
 Container valhalla_nigeria_project-valhalla-1  Removing
 Container valhalla_nigeria_project-valhalla-1  Removed
 Network valhalla_nigeria_project_default  Removing
 Network valhalla_nigeria_project_default  Removed

Valhalla container stopped.
